In [18]:
import pandas as pd
import requests
import boto3
import time
import s3fs
from concurrent.futures import ThreadPoolExecutor

In [5]:
# Initialize the clients for S3 and DynamoDB
from botocore.client import Config
s3 = boto3.client('s3', region_name= "us-east-1",config=Config(signature_version='s3v4'))
s3_resource = boto3.resource('s3')

# Create s3 bucket for Assignment-5
bucket_name = 'final-project-nrel-stations'
s3.create_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'CHCBZ4B1VQZNBMS8',
  'HostId': 'WM8ZaMgAPDm0qS5DSCE7NJsvOq5xnj8Y2Cn9CgfMl8CcZOLo8XjeFckGqMqXAy0neJhTgys+wHw=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'WM8ZaMgAPDm0qS5DSCE7NJsvOq5xnj8Y2Cn9CgfMl8CcZOLo8XjeFckGqMqXAy0neJhTgys+wHw=',
   'x-amz-request-id': 'CHCBZ4B1VQZNBMS8',
   'date': 'Mon, 06 May 2024 03:21:29 GMT',
   'location': '/final-project-nrel-stations',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': '/final-project-nrel-stations'}

In [20]:
# define folder name inside s3 bucket
folder_name = 'station-parquet-files'

In [21]:
def lambda_handler(event,context):

    # Define start time
    stary_time = time.time()

    # Extract relevant attributes from the event
    api_key = event['api_key']
    state=event['state']

    url = f"https://developer.nrel.gov/api/alt-fuel-stations/v1.json?api_key={api_key}&state={state}"
    response = requests.get(url)
    data = response.json()
    
    # Count the number of fuel stations returned
    print(f"Number of fuel stations in {state}: {data['total_results']}")
    
    # Extract the list of fuel stations
    stations = data['fuel_stations']
    stations_df = pd.DataFrame(stations)

    # Upload stations dataframe to S3 bucket as parquet with state initials
    try:
        #stations_df.to_csv('/tmp/stations.csv', index=False)
        #s3.upload_file('/tmp/stations.csv', bucket_name, f'fuel_stations_{state}.csv')
        
        fs = s3fs.S3FileSystem()
        with fs.open(f's3://{bucket_name}/{folder_name}/fuel_stations_{state}.parquet', 'wb') as f:
            stations_df.to_parquet(f, index=False)
        print(f'parquet file uploaded for state {state} to S3')
    except Exception as e:
        print(e)
        print('Error uploading file to S3')    

    # Define end time
    end_time = time.time()

    # Print the time taken to process the data
    print(f"Time taken to get EV data for {state}: {end_time - start_time} seconds")

In [14]:
# Testing lambda locally
api_key = 'uV8N2wWyuRXAFp1hJLIxVlHiq5pBOs1injQxbnjo'
state = 'CA'
event = {'api_key': api_key, 'state': state}
lambda_handler(event, None)

Number of fuel stations in CA: 20052
File uploaded for state CAto S3
